# Convert air pollutant concentrations into AQI score

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import numpy as np
from tqdm import tqdm
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

## Define the table

In [ ]:
pollutants = ['PM2.5','PM10','CO','NO2','SO2','O3']

In [ ]:
AQI_table = {   'AQI': [0, 51, 101, 151, 201, 301, 401, 501],
                'O3': [-1, -1, 0.125, 0.165, 0.205, 0.405, 0.505, 0.605],
                'PM2.5': [0, 12.1, 35.5, 55.5, 150.5, 250.5, 350.5, 500.5],
                'PM10': [0, 55, 155, 255, 355, 425, 505, 605],
                'CO': [0, 4.5, 9.5, 12.5, 15.5, 30.5, 40.5, 50.5],
                'SO2': [0, 36, 76, 186, 305, 605, 805, 1005],
                'NO2': [0, 54, 101, 361, 650, 1250, 1650, 2050]
            }
AQI_table_interval = {
    'AQI': 1,
    'O3': 0.001,
    'PM2.5': 0.1,
    'PM10': 1,
    'CO': 0.1,
    'SO2': 1,
    'NO2': 1
}

## Read and preprocess

In [ ]:
# change the path into the file that you want
data_csvpath = r"../data/MERGED/sensor_camera_20221222_0222/mergedtable.csv"
sensor_df = pd.read_csv(data_csvpath)

In [ ]:
sensor_df.describe()

In [ ]:
sensor_df.drop(['PM1.0'], axis=1, inplace=True)

In [ ]:
sensor_df.Direction = sensor_df.Direction.astype(pd.StringDtype())

In [ ]:
sensor_df['Datetime'] = pd.to_datetime(sensor_df['Datetime'])

In [ ]:
sensor_df_copy = sensor_df.copy(deep=True)

In [ ]:
sensor_df.Datetime = sensor_df.Datetime.dt.round('H')

In [ ]:
sensor_df.Datetime

## Group by sensor and hour

In [ ]:
sensor_groupby = sensor_df.groupby(by=[ sensor_df.SensorCode, sensor_df.Datetime ])

In [ ]:
sensor_hour_df = sensor_groupby.mean()
sensor_hour_df['dt'] = sensor_hour_df.index.get_level_values(1).values
sensor_hour_df.head()
sensor_hour_df.drop(['WeekDay'], axis=1, inplace=True)
sensor_hour_df.describe()

In [ ]:
# ppm -> ppb
# sensor_hour_df['SO2'] *= 1000   
# sensor_hour_df['NO2'] *= 1000

In [ ]:
sensor_hour_df['O3'] = sensor_hour_df['O3'].round(3)
sensor_hour_df['PM2.5'] = sensor_hour_df['PM2.5'].round(1)
sensor_hour_df['PM10'] = sensor_hour_df['PM10'].round()
sensor_hour_df['CO'] = sensor_hour_df['CO'].round(1)
sensor_hour_df['SO2'] = sensor_hour_df['SO2'].round(0)
sensor_hour_df['NO2'] = sensor_hour_df['NO2'].round(0)

In [ ]:
sensor_hour_df[ ['O3', 'PM2.5', 'PM10', 'CO', 'NO2', 'SO2'] ].describe()

https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf

Following the 2015 format of Ozone measurement.

## Calculating moving averages for each pollutant

Following the organizers' note about AQI conversion, we shall use only 1-hour averages, and still use the same table for each pollutant as if they are calculated in specific timescale. 

Calculate the rest: 
- O3 (ppm) 1-hour 
- PM2.5 (μg/m3) 1-hour  (orginially 24h)
- PM10 (μg/m3) 1-hour  (originally 24h)
- CO (ppm) 1-hour       (originally 8h)
- SO2 (ppb) 1-hour 
- NO2 (ppb) 1-hour.

In [ ]:
sensor_hour_df.dropna(inplace=True)
sensor_hour_df.shape

In [ ]:
def convert_to_AQI(series, bps, delta_bps, aqi):
    output = series.copy(deep=True)
    ok = False

    # search for minimum value of concentration
    min_idx = 99999
    max_idx = -1
    for i in range(len(bps)):
        if (bps[i] >= 0):
            min_idx = i
            break
    for i in range(len(bps)):
        if (bps[-i - 1] >= 0):
            max_idx = len(bps) - i - 1
            break
    print(min_idx, max_idx)
    
    for i in tqdm(range(series.shape[0])):

        Cp = series.iloc[i]
        if (Cp < bps[min_idx]):
            output.iloc[i] = 0
            continue
        elif (Cp >= bps[max_idx]):
            output.iloc[i] = aqi[max_idx]
            #print(Cp, output.iloc[i])
            continue

        for j in range(min_idx, len(bps) - 1):
            if (Cp < bps[j+1]):   
                (BPh, BPl, Ih, Il) = bps[j + 1] - delta_bps, bps[j], aqi[j+1]-1, aqi[j] 
                output.iloc[i] = (Ih - Il)*(Cp - BPl) / (BPh - BPl) + Il
                if (output.iloc[i] < 0) or (output.iloc[i] > 1000):
                    print((BPh, BPl, Ih, Il), Cp)
            #print(output.iloc[i])
                break

    return output

In [ ]:
for key in AQI_table:
    if key == 'AQI': continue
    print(key)
    sensor_hour_df['AQI_' + key] = convert_to_AQI(sensor_hour_df[key], AQI_table[key], AQI_table_interval[key], AQI_table['AQI'])

In [ ]:
aqi_columns = ['AQI_O3','AQI_NO2', 'AQI_SO2','AQI_CO', 'AQI_PM2.5', 'AQI_PM10']

In [ ]:
sensor_hour_df[ aqi_columns ].describe()

In [ ]:
import datetime
import json

output_path = '../data/AQI/' + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_path, exist_ok=True)

sensor_hour_df.to_csv(os.path.join(output_path, 'AQI.csv'))
with open(os.path.join(output_path, 'setting.json'), 'w') as f:
    f.write( json.dumps( {'input_path': data_csvpath } )  )